## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**2362. Generate the Invoice (Hard)**

**Table: Products**

| Column Name | Type |
|-------------|------|
| product_id  | int  |
| price       | int  |

product_id contains unique values.
Each row in this table shows the ID of a product and the price of one unit.
 
**Table: Purchases**

| Column Name | Type |
|-------------|------|
| invoice_id  | int  |
| product_id  | int  |
| quantity    | int  |

(invoice_id, product_id) is the primary key (combination of columns with unique values) for this table.
Each row in this table shows the quantity ordered from one product in an invoice. 
 
**Write a solution to show the details of the invoice with the highest price. If two or more invoices have the same price, return the details of the one with the smallest invoice_id.**

Return the result table in any order.

The result format is shown in the following example.

**Example 1:**

**Input:** 

**Products table:**
| product_id | price |
|------------|-------|
| 1          | 100   |
| 2          | 200   |

**Purchases table:**
| invoice_id | product_id | quantity |
|------------|------------|----------|
| 1          | 1          | 2        |
| 3          | 2          | 1        |
| 2          | 2          | 3        |
| 2          | 1          | 4        |
| 4          | 1          | 10       |

**Output:** 
| product_id | quantity | price |
|------------|----------|-------|
| 2          | 3        | 600   |
| 1          | 4        | 400   |

**Explanation:** 
- Invoice 1: price = (2 * 100) = $200
- Invoice 2: price = (4 * 100) + (3 * 200) = $1000
- Invoice 3: price = (1 * 200) = $200
- Invoice 4: price = (10 * 100) = $1000

The highest price is $1000, and the invoices with the highest prices are 2 and 4. We return the details of the one with the smallest ID, which is invoice 2.

In [0]:
products_data_2362 = [
    (1, 100),
    (2, 200),
]

products_columns_2362 = ["product_id", "price"]
products_df_2362 = spark.createDataFrame(products_data_2362, products_columns_2362)
products_df_2362.show()

purchases_data_2362 = [
    (1, 1, 2),
    (3, 2, 1),
    (2, 2, 3),
    (2, 1, 4),
    (4, 1, 10),
]

purchases_columns_2362 = ["invoice_id", "product_id", "quantity"]
purchases_df_2362 = spark.createDataFrame(purchases_data_2362, purchases_columns_2362)
purchases_df_2362.show()


+----------+-----+
|product_id|price|
+----------+-----+
|         1|  100|
|         2|  200|
+----------+-----+

+----------+----------+--------+
|invoice_id|product_id|quantity|
+----------+----------+--------+
|         1|         1|       2|
|         3|         2|       1|
|         2|         2|       3|
|         2|         1|       4|
|         4|         1|      10|
+----------+----------+--------+



In [0]:
invoice_df_2362 = purchases_df_2362.join(products_df_2362, on="product_id")\
                        .withColumn("price", col("price") * col("quantity"))        

In [0]:
total_invoice_df_2362 = invoice_df_2362\
                            .groupBy("invoice_id").agg(sum("price").alias("total_price"))

In [0]:
max_invoice_id_2362 = total_invoice_df_2362\
                            .orderBy(desc("total_price"), asc("invoice_id")).limit(1)\
                                .select("invoice_id")

In [0]:
invoice_df_2362\
    .join(max_invoice_id_2362, on="invoice_id")\
        .select("product_id", "quantity", "price").show()

+----------+--------+-----+
|product_id|quantity|price|
+----------+--------+-----+
|         2|       3|  600|
|         1|       4|  400|
+----------+--------+-----+

